# Ingesta en la Capa Gold

In [0]:
# #Se importan las librerias

from pyspark.sql.functions import year,month,avg,col

In [0]:
# se lee la tabla de la capa de silver

df = spark.read.table("weather.silver.silver_weather")

In [0]:
#Se crea un dataframe con el promedio de precipitacion, temperatura maxima, temperatura minima y velocidad del viento por location, year y month

clima_promedio = (df
    .select(
        "location",
        year("date").alias("year"),
        month("date").alias("month"),
        "precipitation",
        "temp_max",
        "temp_min",
        "wind"
    )
    .groupBy("location", "year", "month").agg(
    avg("precipitation").alias("avg_precipitacion"),
    avg("temp_max").alias("avg_temp_max"),
    avg("temp_min").alias("avg_temp_min"),
    avg("wind").alias("avg_wind")
    )
    )

In [0]:
# Se crea un dataframe que tenga los climas extremos
#- Temperatura maxima mayor a 35 grados centigrados
#- Temperatura minima menor a -10 grados centigrados
#- Precipitacion mayor a 50 mm
#- Velocidad del viento mayor a 10 m/s

climas_extremos = df.filter(
    (col("temp_max") > 35) |
    (col("temp_min") < -10) |
    (col("precipitation") > 50)|
    (col("wind") > 10)
)


In [0]:
# Se escribe los 2 Dataframe en la capa de gold
clima_promedio.write.mode("overwrite").saveAsTable("Weather.gold.gold_weather_summary")
climas_extremos.write.mode("overwrite").saveAsTable("Weather.gold.gold_extreme_weather")